In [12]:
# Please BE VERY CAREFUL, this will link your entire drive.
# So don't edit code, except the one that says 'Customize the following options',
# or you might mess up your files.
# IF YOU DO NO WANT TO LINK DRIVE, please see below for an alternative!
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [13]:
in_path = '/gdrive/MyDrive/anime/in'
out_path = '/gdrive/MyDrive/anime/out/'

In [ ]:
!pip install torchvision Pillow numpy

In [14]:
from google.colab import files
from pathlib import Path
extensions = ["png", "jpeg", "jpg"]

def find_files(in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out
files = [str(f) for f in find_files(in_path)]
print(files)

['/gdrive/MyDrive/anime/in/1517698446560.jpeg', '/gdrive/MyDrive/anime/in/GIULIO.jpeg', '/gdrive/MyDrive/anime/in/BLUE.jpeg', '/gdrive/MyDrive/anime/in/Anna.jpeg']


In [ ]:
import torch
import PIL
from io import BytesIO

images = [ PIL.Image.open(f).convert("RGB") for f in files ]
#for ima in images:
#  display(ima)

In [29]:

# https://github.com/bryandlee/animegan2-pytorch
face2paint = torch.hub.load(
    'bryandlee/animegan2-pytorch:main', 'face2paint', 
    # cpu, cuda, xpu, mkldnn, opengl, opencl, ideep, hip, ve, ort, mlc, xla, lazy, vulkan, meta, hpu
    size=512, device="cpu"
)

Using cache found in /root/.cache/torch/hub/bryandlee_animegan2-pytorch_main


In [42]:
def face2paint(model, image, device="cpu", size=512, side_by_side=True):
    from PIL import Image
    from torchvision.transforms.functional import to_tensor, to_pil_image

    def face2paint(
        model: torch.nn.Module,
        img: Image.Image,
        size: int = size,
        side_by_side: bool = True,
        device: str = device,
    ) -> Image.Image:
        w, h = img.size
        s = min(w, h)
        img = img.crop(((w - s) // 2, (h - s) // 2, (w + s) // 2, (h + s) // 2))
        img = img.resize((size, size), Image.LANCZOS)

        with torch.no_grad():
            input = to_tensor(img).unsqueeze(0) * 2 - 1
            output = model(input.to(device)).cpu()[0]

            if side_by_side:
                output = torch.cat([input[0], output], dim=2)

            output = (output * 0.5 + 0.5).clip(0, 1)

        return to_pil_image(output)

    return face2paint(model, image, size, side_by_side, device)

In [ ]:
models = ["celeba_distill","face_paint_512_v1","face_paint_512_v2","paprika"]
models = ["paprika"]
for model in models:
  model = torch.hub.load(
      "bryandlee/animegan2-pytorch:main",
      "generator",
      pretrained=model
  )
  
  #img = PIL.Image.open(images[0]).convert("RGB")
  for img in images:
    out = face2paint(model, img, side_by_side=False)
    display(out)